In [1]:
from web3 import Web3
import os
from dotenv import load_dotenv
import pandas as pd
import prophet
import numpy as np
import datetime as dt
from datetime import timedelta
from prophet import Prophet
from eth_account import Account
from web3.middleware import geth_poa_middleware
from eth_abi import decode
from eth_utils import decode_hex, to_text

import requests
import random
import json

from dune_client.client import DuneClient

import plotly.graph_objs as go

In [2]:
load_dotenv()

ETHERSCAN_KEY = os.getenv("ETHERSCAN_KEY")
COINGECKO_API_KEY = os.getenv("COINGECKO_API_KEY")
DUNE_API_KEY = os.getenv('DUNE_API_KEY')
FLIPSIDE_API_KEY=os.getenv('FLIPSIDE_API_KEY')
dune = DuneClient(DUNE_API_KEY)

FUND_ACCOUNT_ADDRESS = os.getenv('DEX_ADDRESS')

GAS_RESERVE = os.getenv('GAS_RESERVE')
ACCOUNT_ADDRESS = os.getenv("PROTOCOL_CONTROLLER")
PRIVATE_KEY = os.getenv("PROTOCOL_CONTROLLER_KEY")
YIELD_FARM_ADDRESS = os.getenv("YIELD_FARM_ADDRESS")
STAKING_CONTRACT = os.getenv("STAKING_CONTRACT")
SEPOLIA_GATEWAY = os.getenv("SEPOLIA_GATEWAY")

BOT_1_ADDRESS=os.getenv("BOT_1_ADDRESS")
BOT_2_ADDRESS=os.getenv("BOT_2_ADDRESS")

BOT_1_KEY=os.getenv("BOT_1_KEY")
BOT_2_KEY=os.getenv("BOT_2_KEY")

TBTC_CONTRACT_ADDRESS = os.getenv('TEST_BTC')
TETH_CONTRACT_ADDRESS = os.getenv('TETH_WETH')

In [3]:
os.chdir('..')

api = False

In [4]:
from python_scripts.web3_utils import get_balance

In [5]:
abi_path = r'gas_accountant_contracts\contracts\artifacts'
abi_paths = []  # Assuming GAS_ACCOUNTANT_ABI_PATH is predefined

for file in os.listdir(abi_path):
    if file.endswith('.json') and "metadata" in file:  # Exclude metadata files
        abi_paths.append(os.path.join(abi_path, file))  # Add full path

print(abi_paths)  # Debug: Check the final list

abis = {}

for path in abi_paths:
    filename = os.path.basename(path)  # Extract filename (e.g., "YieldVault.json")
    name = os.path.splitext(filename)[0]  # Remove .json extension (e.g., "YieldVault")

    with open(path, "r") as file:
        abis[name] = json.load(file)  # Use name as key

print(abis)  # Debug output

['gas_accountant_contracts\\contracts\\artifacts\\BatchTransfer_metadata.json', 'gas_accountant_contracts\\contracts\\artifacts\\GasReserve_metadata.json', 'gas_accountant_contracts\\contracts\\artifacts\\IGasReserve_metadata.json', 'gas_accountant_contracts\\contracts\\artifacts\\ILiquidStaking_metadata.json', 'gas_accountant_contracts\\contracts\\artifacts\\LiquidStaking_metadata.json', 'gas_accountant_contracts\\contracts\\artifacts\\StETHStrategy_metadata.json', 'gas_accountant_contracts\\contracts\\artifacts\\TestBTC_metadata.json', 'gas_accountant_contracts\\contracts\\artifacts\\TestETH_metadata.json', 'gas_accountant_contracts\\contracts\\artifacts\\YieldVault_metadata.json']
{'BatchTransfer_metadata': {'compiler': {'version': '0.8.26+commit.8a97fa7a'}, 'language': 'Solidity', 'output': {'abi': [{'inputs': [{'internalType': 'address[]', 'name': 'recipients', 'type': 'address[]'}, {'internalType': 'uint256[]', 'name': 'amounts', 'type': 'uint256[]'}], 'name': 'batchSendETH', 'ou

In [6]:
def get_token_price(token='0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'):
    url = f"https://api.coingecko.com/api/v3/simple/token_price/ethereum?contract_addresses={token}&vs_currencies=usd"

    headers = {
        "accept": "application/json",
        "x-cg-demo-api-key": COINGECKO_API_KEY
    }

    response = requests.get(url, headers=headers)

    

    eth_data = response.json()

    eth_df = pd.DataFrame(eth_data)
    eth_usd = eth_df[f'{token}'].values[0]

    print(eth_usd)

    return eth_usd

In [7]:
W3_BOT1 = Web3(Web3.HTTPProvider(SEPOLIA_GATEWAY))

ACCOUNT_BOT_1 = Account.from_key(BOT_1_KEY)

W3_BOT1.eth.default_account = ACCOUNT_BOT_1.address
W3_BOT1.middleware_onion.inject(geth_poa_middleware, layer=0)

In [8]:
print(f"✅ Web3 Connection Check: {W3_BOT1.is_connected()}")
print(f"✅ Using Account: {Account.from_key(PRIVATE_KEY).address}")
print(f"✅ Next Nonce: {W3_BOT1.eth.get_transaction_count(Account.from_key(PRIVATE_KEY).address, 'pending')}")


✅ Web3 Connection Check: True
✅ Using Account: 0x2102240d1A36a9DC9F3A4d07eE9251cb723ACa89
✅ Next Nonce: 83


In [9]:
W3_BOT2 = Web3(Web3.HTTPProvider(SEPOLIA_GATEWAY))

ACCOUNT_BOT_2 = Account.from_key(BOT_2_KEY)

W3_BOT2.eth.default_account = ACCOUNT_BOT_2.address
W3_BOT2.middleware_onion.inject(geth_poa_middleware, layer=0)

In [10]:
TOKEN_CONTRACTS = {'tbtc':TBTC_CONTRACT_ADDRESS,
                   'teth':TETH_CONTRACT_ADDRESS }
TOKEN_CONTRACTS = {token.upper(): address for token, address in TOKEN_CONTRACTS.items()}

In [11]:
TOKEN_CONTRACTS

{'TBTC': '0x86Ce29079Cc2b017Dbf3285fce40c911000a0F45',
 'TETH': '0x3fc47744B859e08c59b71c883bDC204eBA100B60'}

In [12]:
BOT_1_BALANCE  = get_balance(W3_BOT1,ACCOUNT_BOT_1,TOKEN_CONTRACTS)
BOT_2_BALANCE  = get_balance(W3_BOT2,ACCOUNT_BOT_2,TOKEN_CONTRACTS)
print(f'original_balances: {BOT_1_BALANCE} \n {BOT_2_BALANCE}')


Balances for account <eth_account.signers.local.LocalAccount object at 0x000001CF1B2255D0>: {'TBTC': 10.30209870380667, 'TETH': 123.96270172356736}
Balances for account <eth_account.signers.local.LocalAccount object at 0x000001CF1B6316D0>: {'TBTC': 0.22623028869108427, 'TETH': 17.58182746611581}
original_balances: {'TBTC': 10.30209870380667, 'TETH': 123.96270172356736} 
 {'TBTC': 0.22623028869108427, 'TETH': 17.58182746611581}


In [13]:
prices = {
    "TBTC":get_token_price('0x2260fac5e5542a773aa44fbcfedf7c193bc2c599'),
    'TETH':get_token_price()
   
}

prices

96811
2704.82


{'TBTC': 96811, 'TETH': 2704.82}

In [14]:
def convert_to_usd(balances, prices):
    """
    Convert token balances to their USD equivalent using token prices.

    Parameters:
    - balances (dict): Dictionary of token balances.
    - prices (dict): Dictionary of token prices.

    Returns:
    - dict: Dictionary of token balances converted to USD.
    """
    # Convert token keys to upper case for consistency
    balances = {token: balance for token, balance in balances.items()}

    print(f'balances: {balances.keys()}')
    print(f'TOKEN_CONTRACTS.keys(): {TOKEN_CONTRACTS.keys()}')

    for token in TOKEN_CONTRACTS.keys():
        if f"{token}" not in prices:
            print(f"Missing price for token: {token}")

    usd_balances = {
        token: balances[token] * prices[f"{token}"]
        for token in TOKEN_CONTRACTS.keys()
        if f"{token}" in prices
    }
    return usd_balances

In [15]:
def get_wallet_state(original_balances,prices):
    # Fetch original holdings dynamically based on TOKEN_CONTRACTS
    original_holdings = {
        token: float(original_balances[token])
        for token in TOKEN_CONTRACTS.keys() if token in original_balances
    }

    print(f'initial prices for USD conversion: {prices}')
    print(f'initial balances used for USD conversion: {original_holdings}')

    # Convert balances to USD
    balances_in_usd = convert_to_usd(original_holdings, prices)
    initial_portfolio_balance = sum(balances_in_usd.values())

    # Calculate compositions dynamically
    print(f'balances_in_usd.items(): {balances_in_usd.items()}')

    comp_dict = {
        f"{token} comp": balance_usd / initial_portfolio_balance
        for token, balance_usd in balances_in_usd.items()
    }

    today_utc = dt.datetime.now(dt.timezone.utc) 
    formatted_today_utc = today_utc.strftime('%Y-%m-%d %H:00:00')

    comp_dict["date"] = formatted_today_utc

    print(f'Composition dictionary: {comp_dict}')

    return comp_dict, balances_in_usd, initial_portfolio_balance


In [16]:
BOT_1_BALANCE

{'TBTC': 10.30209870380667, 'TETH': 123.96270172356736}

In [17]:
prices

{'TBTC': 96811, 'TETH': 2704.82}

In [18]:
BOT_1_STATE = get_wallet_state(BOT_1_BALANCE,prices)

initial prices for USD conversion: {'TBTC': 96811, 'TETH': 2704.82}
initial balances used for USD conversion: {'TBTC': 10.30209870380667, 'TETH': 123.96270172356736}
balances: dict_keys(['TBTC', 'TETH'])
TOKEN_CONTRACTS.keys(): dict_keys(['TBTC', 'TETH'])
balances_in_usd.items(): dict_items([('TBTC', 997356.4776142276), ('TETH', 335296.7948759395)])
Composition dictionary: {'TBTC comp': 0.748399075890602, 'TETH comp': 0.25160092410939805, 'date': '2025-02-07 01:00:00'}


In [19]:
BOT_2_STATE = get_wallet_state(BOT_2_BALANCE,prices)

initial prices for USD conversion: {'TBTC': 96811, 'TETH': 2704.82}
initial balances used for USD conversion: {'TBTC': 0.22623028869108427, 'TETH': 17.58182746611581}
balances: dict_keys(['TBTC', 'TETH'])
TOKEN_CONTRACTS.keys(): dict_keys(['TBTC', 'TETH'])
balances_in_usd.items(): dict_items([('TBTC', 21901.58047847256), ('TETH', 47555.67856689937)])
Composition dictionary: {'TBTC comp': 0.3153245719668505, 'TETH comp': 0.6846754280331494, 'date': '2025-02-07 01:00:00'}


In [20]:
def normalize_portfolio(portfolio):
    total = np.sum(portfolio)
    if total == 0:
        # If the total is zero, avoid division by zero and return an equally distributed portfolio
        return np.ones_like(portfolio) / len(portfolio)
    return portfolio / total

In [21]:
BOT_1_PORTFOLIO_ACTION = np.random.rand(2)
BOT_1_PORTFOLIO_ACTION = normalize_portfolio(BOT_1_PORTFOLIO_ACTION) 

In [22]:
BOT_2_PORTFOLIO_ACTION = np.random.rand(2)
BOT_2_PORTFOLIO_ACTION = normalize_portfolio(BOT_2_PORTFOLIO_ACTION) 

In [23]:
BOT_1_ACTION_DF = pd.DataFrame(columns=['TETH','TBTC'],data=BOT_1_PORTFOLIO_ACTION.reshape(1, -1))
BOT_1_ACTION_DF

,TETH,TBTC
0,0.793294,0.206706


In [24]:
# Get new compositions dynamically from model actions
new_compositions = {
    token: float(BOT_1_ACTION_DF.iloc[-1][f"{token}"]) for token in TOKEN_CONTRACTS
}

print(f'new compositions: {new_compositions}')

# Calculate total portfolio value
total_value = sum(
    BOT_1_BALANCE[token] * prices[f"{token}"] for token in TOKEN_CONTRACTS
)

target_balances = {
    token: total_value * new_compositions.get(token, 0) / prices[f"{token}"]
    for token in TOKEN_CONTRACTS
}

new compositions: {'TBTC': 0.20670603456505532, 'TETH': 0.7932939654349447}


In [25]:
token_1_rebal_info = {
    "new compositions": new_compositions,
    "prices": prices,
    "initial holdings": BOT_1_BALANCE,
    "account address": BOT_1_ADDRESS,
    "target balances": target_balances,
    **{
        f"{token} bal usd": BOT_1_BALANCE[token] * prices[f"{token}"]
        for token in TOKEN_CONTRACTS
    },
    "portfolio balance": total_value
}

In [26]:
token_1_rebal_info

{'new compositions': {'TBTC': 0.20670603456505532, 'TETH': 0.7932939654349447},
 'prices': {'TBTC': 96811, 'TETH': 2704.82},
 'initial holdings': {'TBTC': 10.30209870380667, 'TETH': 123.96270172356736},
 'account address': '0x21efbeE92E732D9d87Ae7b67E0aae7a972bd23F8',
 'target balances': {'TBTC': 2.845415019022493, 'TETH': 390.8525517718667},
 'TBTC bal usd': 997356.4776142276,
 'TETH bal usd': 335296.7948759395,
 'portfolio balance': 1332653.272490167}

In [27]:
erc20_abi = [
    {
        "constant": False,
        "inputs": [
            {"name": "_to", "type": "address"},
            {"name": "_value", "type": "uint256"}
        ],
        "name": "transfer",
        "outputs": [{"name": "", "type": "bool"}],
        "type": "function"
    }
]

In [28]:
def get_contract_address(token):
    """Get contract address based on token name."""
    if token == 'TBTC':
        return TBTC_CONTRACT_ADDRESS
    elif token == 'TETH':
        return TETH_CONTRACT_ADDRESS
    else:
        raise ValueError(f"Unknown token: {token}")

In [29]:
from web3.exceptions import TransactionNotFound,TimeExhausted
import asyncio
import math
import traceback
import aiohttp


In [30]:
async def wait_for_transaction(w3,tx_hash, retries=30, delay=60):
    """Wait for transaction confirmation with detailed debugging."""
    print(f"Waiting for transaction receipt: {tx_hash.hex()}")
    for attempt in range(retries):
        try:
            receipt = w3.eth.get_transaction_receipt(tx_hash)
            print(f"Transaction receipt: {receipt}")
            if receipt and receipt['status'] == 1:
                print(f"Transaction {tx_hash.hex()} confirmed successfully.")
                return receipt
            elif receipt and receipt['status'] == 0:
                raise Exception(f"Transaction {tx_hash.hex()} failed on-chain.")
        except TransactionNotFound:
            print(f"Transaction {tx_hash.hex()} not yet confirmed. Attempt {attempt + 1}/{retries}. Retrying...")
        except TimeExhausted:
            print("Transaction confirmation timed out.")
            break
        await asyncio.sleep(delay)
    raise Exception(f"Transaction {tx_hash.hex()} failed to confirm within {retries} attempts.")

In [31]:
async def transfer_tokens(w3_instance, private_key, token, amount, recipient_address):
    """Ensure the correct Web3 instance is used for transactions."""
    print(f"Starting transfer: token={token}, amount={amount}, recipient={recipient_address}")

    sender_address = Account.from_key(private_key).address
    print(f"Sender Address: {sender_address}")

    contract_address = get_contract_address(token)
    amount_wei = int(amount * 10**18)

    # ERC20 ABI
    erc20_abi = [
        {
            "constant": False,
            "inputs": [{"name": "_to", "type": "address"}, {"name": "_value", "type": "uint256"}],
            "name": "transfer",
            "outputs": [{"name": "", "type": "bool"}],
            "type": "function"
        }
    ]

    contract = w3_instance.eth.contract(address=Web3.to_checksum_address(contract_address), abi=erc20_abi)

    # ✅ Get latest nonce (fixes "nonce too low" issue)
    nonce = w3_instance.eth.get_transaction_count(sender_address, "pending")
    print(f"Using nonce: {nonce}")

    # ✅ Fetch gas price
    gas_price = max(w3_instance.eth.gas_price, w3_instance.to_wei("20", "gwei"))

    # ✅ Estimate gas
    try:
        estimated_gas = contract.functions.transfer(
            Web3.to_checksum_address(recipient_address), amount_wei
        ).estimate_gas({'from': sender_address})
    except Exception as e:
        print(f"Gas estimation failed: {e}")
        return None

    # ✅ Build transaction
    tx = contract.functions.transfer(
        Web3.to_checksum_address(recipient_address),
        amount_wei
    ).build_transaction({
        'chainId': w3_instance.eth.chain_id,
        'gas': estimated_gas,
        'gasPrice': gas_price,
        'nonce': nonce,
    })

    print(f"Transaction details: {tx}")

    # ✅ Sign transaction
    signed_tx = w3_instance.eth.account.sign_transaction(tx, private_key=private_key)
    print(f"Signed transaction hash: {signed_tx.hash.hex()}")

    # ✅ Send transaction
    try:
        tx_hash = w3_instance.eth.send_raw_transaction(signed_tx.rawTransaction)
        print(f"Transaction sent: {tx_hash.hex()}")
        return tx_hash.hex()
    except Exception as e:
        print(f"Error sending transaction: {e}")
        return None


In [32]:
import json
import numpy as np
import aiohttp
import asyncio

async def send_rebalance_request(web3, token, amount_to_send, recipient_address, prices, initial_holdings, new_compositions):
    """Send rebalance request to the DEX app's endpoint."""
    url = 'http://127.0.0.1:5001/rebalance'

    # Convert all NumPy types to Python native types
    def convert_numpy(obj):
        if isinstance(obj, np.integer):
            return int(obj)  # Convert np.int64 → int
        elif isinstance(obj, np.floating):
            return float(obj)  # Convert np.float64 → float
        elif isinstance(obj, dict):
            return {k: convert_numpy(v) for k, v in obj.items()}  # Convert dict values
        elif isinstance(obj, list):
            return [convert_numpy(i) for i in obj]  # Convert list elements
        return obj  # Return original if not NumPy type

    rebalance_data = {
        'recipient_address': web3.to_checksum_address(recipient_address),
        'prices': convert_numpy(prices),
        'initial_holdings': convert_numpy(initial_holdings),
        'new_compositions': convert_numpy(new_compositions)
    }

    print(f"Sending rebalance request to {url} with data: {rebalance_data}")

    try:
        async with aiohttp.ClientSession(timeout=aiohttp.ClientTimeout(total=300)) as session:
            async with session.post(url, json=rebalance_data) as response:
                if response.status == 200:
                    print(f"Rebalance response: {await response.json()}")
                else:
                    print(f"Error: Received status {response.status} with response: {await response.text()}")
    except asyncio.TimeoutError:
        print(f"Timeout error when sending rebalance request for token: {token}")
    except aiohttp.ClientError as e:
        print(f"Error sending rebalance request: {e}, data: {rebalance_data}")


In [33]:
async def send_balances_to_fund(web3, contract_abi, private_key, initial_holdings, target_balances, prices, new_compositions,ACCOUNT_ADDRESS):
    print(f"Starting send back balance function...")
    print("Current balances:", initial_holdings)
    print("Target balances:", target_balances)

    processed_tokens = set()
    needs_rebalance = False  # Track if we need to send a rebalance request

    for token, target_balance in target_balances.items():
        if token in processed_tokens:
            print(f"Token {token} already processed. Skipping...")
            continue

        processed_tokens.add(token)

        current_balance = initial_holdings.get(token, 0)
        print(f"Token: {token}, current balance: {current_balance}")
        amount_to_adjust = current_balance - target_balance
        amount_to_adjust = math.floor(amount_to_adjust * 10**6) / 10**6
        print(f"Token: {token}, clipped amount to adjust: {amount_to_adjust}")

        if math.isclose(amount_to_adjust, 0, abs_tol=1e-6):
            print(f"Skipping token {token} with negligible adjustment: {amount_to_adjust}")
            continue

        try:
            if amount_to_adjust > 0:
                # Handle excess tokens individually
                print(f"Sending back {amount_to_adjust} of {token} to the fund.")
                await transfer_tokens(web3,private_key,token=token, amount=amount_to_adjust, recipient_address=FUND_ACCOUNT_ADDRESS)
            elif amount_to_adjust < 0:
                # Mark that we need a rebalance request
                needs_rebalance = True
        except Exception as e:
            print(f"Error processing token {token}: {e}")
            traceback.print_exc()

    # If any token requires rebalance, send a single request for all tokens
    if needs_rebalance:
        try:
            await send_rebalance_request(
                web3=web3,
                token=None,  # Not used
                amount_to_send=None,  # Not used
                recipient_address=ACCOUNT_ADDRESS,
                prices=prices,
                initial_holdings=initial_holdings,
                new_compositions=new_compositions,
            )
        except Exception as e:
            print(f"Error during rebalance request: {e}")
            traceback.print_exc()

    print("Completed sending balances to fund.")

w3.eth.get_transaction_count(ACCOUNT_ADDRESS)

In [34]:
from eth_account import Account

In [35]:
target_balances

{'TBTC': 2.845415019022493, 'TETH': 390.8525517718667}

In [36]:
ACCOUNT_ADDRESS = Account.from_key(PRIVATE_KEY).address

print(f"✅ Your account: {ACCOUNT_ADDRESS}")

# Set default account
W3_BOT1.eth.default_account = ACCOUNT_ADDRESS

✅ Your account: 0x2102240d1A36a9DC9F3A4d07eE9251cb723ACa89


In [37]:
print(f"DEBUG: Web3 Client Version: {W3_BOT1.client_version}")  # ✅ CORRECT
print(f"Web3 Instance Type: {type(W3_BOT1)}")
print(f"Web3 Connected: {W3_BOT1.is_connected()}")
print(f"Web3 Client Version: {getattr(W3_BOT1, 'client_version', 'Not Available')}")
print(f"Web3 Accounts: {W3_BOT1.eth.accounts if W3_BOT1.is_connected() else 'Not Connected'}")


DEBUG: Web3 Client Version: Geth/v1.14.13-stable-eb00f169/linux-amd64/go1.23.5
Web3 Instance Type: <class 'web3.main.Web3'>
Web3 Connected: True
Web3 Client Version: Geth/v1.14.13-stable-eb00f169/linux-amd64/go1.23.5
Web3 Accounts: []


In [38]:
await send_balances_to_fund(W3_BOT1, erc20_abi, BOT_1_KEY, BOT_1_BALANCE, target_balances, prices, new_compositions,BOT_1_ADDRESS)

Starting send back balance function...
Current balances: {'TBTC': 10.30209870380667, 'TETH': 123.96270172356736}
Target balances: {'TBTC': 2.845415019022493, 'TETH': 390.8525517718667}
Token: TBTC, current balance: 10.30209870380667
Token: TBTC, clipped amount to adjust: 7.456683
Sending back 7.456683 of TBTC to the fund.
Starting transfer: token=TBTC, amount=7.456683, recipient=0x24E1F4029BbC228B74fE221f29821CF64366C2Fa
Sender Address: 0x21efbeE92E732D9d87Ae7b67E0aae7a972bd23F8
Using nonce: 37
Transaction details: {'value': 0, 'chainId': 11155111, 'gas': 34863, 'gasPrice': 20000000000, 'nonce': 37, 'to': '0x86Ce29079Cc2b017Dbf3285fce40c911000a0F45', 'data': '0xa9059cbb00000000000000000000000024e1f4029bbc228b74fe221f29821cf64366c2fa000000000000000000000000000000000000000000000000677b75ae69b1b000'}
Signed transaction hash: 0xbaa469a02bce36c77de9df5bf9b6644f100ce163ca10213ced7691a3b539f039
Transaction sent: 0xbaa469a02bce36c77de9df5bf9b6644f100ce163ca10213ced7691a3b539f039
Token: TETH, c

In [39]:
abis.keys()

dict_keys(['BatchTransfer_metadata', 'GasReserve_metadata', 'IGasReserve_metadata', 'ILiquidStaking_metadata', 'LiquidStaking_metadata', 'StETHStrategy_metadata', 'TestBTC_metadata', 'TestETH_metadata', 'YieldVault_metadata'])

In [40]:
controller_w3 = Web3(Web3.HTTPProvider(SEPOLIA_GATEWAY))

ACCOUNT = Account.from_key(PRIVATE_KEY)

controller_w3.eth.default_account = ACCOUNT.address
controller_w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [41]:
abi_file_path = "abi/vault_abi.json"
staking_abi = "abi/staking_abi.json"

abi_paths = [abi_file_path,staking_abi]
abis = {}

for path in abi_paths:
    with open(path, "r") as file:
        abis[path] = json.load(file)

print(abis)

{'abi/vault_abi.json': [{'inputs': [{'internalType': 'address', 'name': '_liquidStaking', 'type': 'address'}, {'internalType': 'address', 'name': '_gasReserve', 'type': 'address'}], 'stateMutability': 'nonpayable', 'type': 'constructor'}, {'inputs': [{'internalType': 'address', 'name': 'spender', 'type': 'address'}, {'internalType': 'uint256', 'name': 'allowance', 'type': 'uint256'}, {'internalType': 'uint256', 'name': 'needed', 'type': 'uint256'}], 'name': 'ERC20InsufficientAllowance', 'type': 'error'}, {'inputs': [{'internalType': 'address', 'name': 'sender', 'type': 'address'}, {'internalType': 'uint256', 'name': 'balance', 'type': 'uint256'}, {'internalType': 'uint256', 'name': 'needed', 'type': 'uint256'}], 'name': 'ERC20InsufficientBalance', 'type': 'error'}, {'inputs': [{'internalType': 'address', 'name': 'approver', 'type': 'address'}], 'name': 'ERC20InvalidApprover', 'type': 'error'}, {'inputs': [{'internalType': 'address', 'name': 'receiver', 'type': 'address'}], 'name': 'ERC

In [42]:
vault_contract = controller_w3.eth.contract(address=YIELD_FARM_ADDRESS, abi=abis['abi/vault_abi.json'])

In [43]:
staking_contract = controller_w3.eth.contract(address=STAKING_CONTRACT, abi=abis['abi/staking_abi.json'])

In [49]:
balance_wei = controller_w3.eth.get_balance(ACCOUNT.address)
print(f"Account Balance: {controller_w3.from_wei(balance_wei, 'ether')} ETH")

# Transaction to Deposit ETH
tx = vault_contract.functions.deposit().build_transaction({
    'from': ACCOUNT.address,
    'value': int(balance_wei*0.25),  # Depositing full balance
    'gas': 200000,
    'gasPrice': controller_w3.eth.gas_price,
    'nonce': controller_w3.eth.get_transaction_count(ACCOUNT.address)
})

# Sign & Send Transaction
signed_tx = controller_w3.eth.account.sign_transaction(tx, PRIVATE_KEY)
tx_hash = controller_w3.eth.send_raw_transaction(signed_tx.rawTransaction)

print(f"Deposit Transaction Sent: {tx_hash.hex()}")

Account Balance: 0.406002785806493003 ETH
Deposit Transaction Sent: 0x0001c2a4e504b0c542cb33c360e37f9f9faa07cbd089a7f93b6b3d9573304d25


In [44]:
earned_rewards = vault_contract.functions.earned(ACCOUNT.address).call()
print(f"Earned Rewards: {earned_rewards / 10**18} stETH")


Earned Rewards: 0.002 stETH


In [45]:
try:
    gas_estimate = staking_contract.functions.claimRewards().estimate_gas({'from': YIELD_FARM_ADDRESS})
    print(f"Estimated Gas: {gas_estimate}")
except Exception as e:
    print(f"Gas estimation failed: {e}")
    gas_estimate = 200000  # Default fallback gas

Estimated Gas: 111159


In [46]:
# ✅ Estimate Gas for Harvest

tx = vault_contract.functions.harvest(gas_estimate).build_transaction({
    'from': ACCOUNT.address,
    'gas': 200000,
    'gasPrice': controller_w3.eth.gas_price,
    'nonce': controller_w3.eth.get_transaction_count(ACCOUNT.address)
})

signed_tx = controller_w3.eth.account.sign_transaction(tx, PRIVATE_KEY)
tx_hash = controller_w3.eth.send_raw_transaction(signed_tx.rawTransaction)

print(f"Harvest Transaction Sent: {tx_hash.hex()}")


Harvest Transaction Sent: 0x577b9e019f157cc00b97457b643f2ff81c6833eafadb7ea80487e2eb2a64aa62


In [47]:
gas_reserve_spent = {
    'hash':tx_hash.hex(),
    'gas used':gas_estimate
}

In [49]:
from diskcache import Cache

In [50]:
base_cache_dir = r'E:\Projects\gas_accountant'
cache = Cache(os.path.join(base_cache_dir, 'cache')) 

In [54]:
def update_gas_reserve_data(live_comp):
    new_data = pd.DataFrame([live_comp])
    historical_data = cache.get(f'gas_reserve_spent', pd.DataFrame())
    historical_data = pd.concat([historical_data, new_data]).reset_index(drop=True)
    historical_data.drop_duplicates(subset='hash', keep='last', inplace=True)
    cache.set(f'gas_reserve_spent', historical_data)

In [55]:
update_gas_reserve_data(gas_reserve_spent)

In [56]:
historical_data = cache.get(f'gas_reserve_spent')

In [57]:
historical_data

,hash,gas used
0,0x577b9e019f157cc00b97457b643f2ff81c6833eafadb...,111159
